# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Terminal command to import the dataset: `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

Inline command to import the dataset: `!mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

In [1]:
# Import the dataset
!mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json

2023-05-19T23:16:51.803-0700	connected to: mongodb://localhost/
2023-05-19T23:16:51.805-0700	dropping: uk_food.establishments
2023-05-19T23:16:54.804-0700	[######################..] uk_food.establishments	37.6MB/39.3MB (95.7%)
2023-05-19T23:16:54.943-0700	[########################] uk_food.establishments	39.3MB/39.3MB (100.0%)
2023-05-19T23:16:54.943-0700	39779 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# Confirm presence of new database
db_names = mongo.list_database_names()
db_to_check = "uk_food"
if db_to_check in db_names:
    print(f"Database found: {db_to_check}")
else:
    print("Database not found")

Database found: uk_food


In [5]:
# Assign new database to a variable
db = mongo["uk_food"]

In [6]:
# List the collections in the new database
db.list_collection_names()

['establishments']

In [7]:
# Print a document from the establishments collection
one_doc = db.establishments.find_one()
pprint(one_doc)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('646865d3076073219f6a8a7e'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [8]:
# Assign the collection to a variable
establishments = db["establishments"]

# Part 2: Update the Database

Insert the new halal restaurant "Penang Flavours" into the database.

In [9]:
# Create a dictionary for the new restaurant
penang_dict = {"BusinessName":"Penang Flavours",
               "BusinessType":"Restaurant/Cafe/Canteen",
               "BusinessTypeID":"",
               "AddressLine1":"Penang Flavours",
               "AddressLine2":"146A Plumstead Rd",
               "AddressLine3":"London",
               "AddressLine4":"",
               "PostCode":"SE18 7DY",
               "Phone":"",
               "LocalAuthorityCode":"511",
               "LocalAuthorityName":"Greenwich",
               "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
               "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
               "scores":{"Hygiene":"",
                         "Structural":"",
                         "ConfidenceInManagement":""},
               "SchemeType":"FHRS",
               "geocode":{"longitude":"0.08384000",
                          "latitude":"51.49014200"},
               "RightToReply":"",
               "Distance":4623.9723280747176,
               "NewRatingPending":True}

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_dict)

In [11]:
# Confirm insertion
query = {"BusinessName": "Penang Flavours"}
results = establishments.find_one(query)
pprint(results)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64687a738e7c627e31c1ef94'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessType":1,
          "BusinessTypeID":1}
results = establishments.find_one(query,fields)
business_type = results["BusinessType"]
business_type_id = results["BusinessTypeID"]
pprint(results)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('646865d3076073219f6a8a7e')}


3. Update the new restaurant's `BusinessTypeID`

In [14]:
# Update the new restaurant's BusinessTypeID
query = {"BusinessName":"Penang Flavours"}
fields = {"$set":{"BusinessTypeID":business_type_id}}
establishments.update_one(query,fields)

In [15]:
# Confirm the update
query = {"BusinessName":"Penang Flavours"}
fields = {"BusinessTypeID":10}
results = establishments.find_one(query,fields)
pprint(results)

{'BusinessTypeID': 1, '_id': ObjectId('64687a738e7c627e31c1ef94')}


4. Remove any establishments within the Dover Local Authority from the database

In [16]:
# Count the documents with LocalAuthorityName Dover
query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query)

994

In [17]:
# Delete documents with LocalAuthorityName Dover
query = {"LocalAuthorityName":"Dover"}
establishments.delete_many(query)

In [18]:
# Confirm deletion
query = {"LocalAuthorityName":"Dover"}
establishments.count_documents(query)

0

In [19]:
# Verify that all documents weren't deleted
result = establishments.find_one()
pprint(result)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('646865d3076073219f6a8d62'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Update data types

Convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Check the data types for latitude and longitude
results = establishments.find_one()
latitude = results["geocode"]["latitude"]
longitude = results["geocode"]["longitude"]
pprint(type(latitude))
pprint(type(longitude))

<class 'str'>
<class 'str'>


In [24]:
# Convert latitude and longitude to double
query = {}

update = [{"$set":{"geocode.latitude":{"$toDouble":"$geocode.latitude"}}}]
establishments.update_many(query,update)

update = [{"$set":{"geocode.longitude":{"$toDouble":"$geocode.longitude"}}}]
establishments.update_many(query,update)

Convert `RatingValue` to integer.

In [25]:
# Check the data type for RatingValue
results = establishments.find_one()
rating_value = results["RatingValue"]
pprint(type(rating_value))

<class 'str'>


In [27]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection",
               "Awaiting Inspection",
               "AwaitingPublication",
               "Pass",
               "Exempt"]
query = {"RatingValue":{"$in":non_ratings}}
update = [{"$set":{"RatingValue":None}}]
establishments.update_many(query,update)

In [28]:
# Convert RatingValue to integer
query = {}
update = [{"$set":{"RatingValue":{"$toInt":"$RatingValue"}}}]
establishments.update_many(query,update)